# Reinforcement Learning control strategies for Electric Vehicles fleet Virtual Power Plants
Thesis based on the development of a RL agent that manages a VPP through EVs charging stations in an household environment. Main optimization objectives of the VPP are: Valley filling, peak shaving and zero resulting load over time. Main action performed to reach objectives are: storage of Renewable energy resources and power push in the grid at high demand times. The development of the Virtual Power Plant environment is based on the ELVIS (Electric Vehicles Infrastructure Simulator) open library from DAI-Labor: https://github.com/dailab/elvis The thesis code is currently available at: (https://github.com/francescomaldonato/RL_VPP_Thesis)

Author: Francesco Maldonato

## Algorithms results' plotting Notebook

In [13]:
%%capture
!pip install scikit-learn==1.0.2
#!pip install scipy
!pip install plotly==5.9.0

In [14]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/data/algorithms_results/
%ls

Cloning into 'RL_VPP_Thesis'...
remote: Enumerating objects: 503, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 503 (delta 55), reused 107 (delta 54), pack-reused 393
Receiving objects: 100% (503/503), 184.47 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (204/204), done.
Checking out files: 100% (226/226), done.
/content/RL_VPP_Thesis/data/algorithms_results/RL_VPP_Thesis/data/algorithms_results
algorithms_graphs/              Experiments_testing_plots.ipynb
Algorithms_results_plots.ipynb  Experiments_validator_plots.ipynb
algorithms_results_table/


In [15]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.linalg
from sklearn.linear_model import LinearRegression
#from IPython.display import Image

In [16]:
results_data = pd.read_csv('algorithms_results_table/EV_experiments_testing.csv')
results_data = results_data.iloc[: , 1:]
results_data['underconsume'].astype(float)
results_data['overconsume'].astype(float)
results_data['overcost'].astype(float)
results_data['av_EV_energy_left'].astype(float)
results_data['cumulative_reward'].astype(float)
results_data.info()
results_data.tail(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               66 non-null     object 
 1   underconsume       66 non-null     float64
 2   overconsume        66 non-null     float64
 3   overcost           66 non-null     float64
 4   av_EV_energy_left  66 non-null     float64
 5   cumulative_reward  66 non-null     float64
dtypes: float64(5), object(1)
memory usage: 3.2+ KB


,Name,underconsume,overconsume,overcost,av_EV_energy_left,cumulative_reward
60,10_EVs,13349.291090,3732.743527,135.344791,72.643636,213189.316120
61,10_EVs,13553.569131,3718.279571,136.238523,71.511614,212506.653889
62,10_EVs,13386.278370,3814.699691,138.808435,72.563522,212169.591302
63,10_EVs,13409.985534,4123.641650,157.119521,72.805412,206174.566321
64,10_EVs,13929.257537,4018.942940,146.332679,71.831817,201562.735483
65,10_EVs_mean,13573.092000,3719.518800,136.330170,71.806984,212277.600000


In [17]:
results_data_=results_data.drop("Name", axis=1)
results_data_=results_data_.drop("overcost", axis=1)
results_data_=results_data_.drop("cumulative_reward", axis=1) 
data = results_data_.values

underc_min = results_data_["underconsume"].values[np.argmin(results_data_["underconsume"].values)]
overc_min = results_data_["overconsume"].values[np.argmin(results_data_["overconsume"].values)]
underc_max = results_data_["underconsume"].values[np.argmax(results_data_["underconsume"].values)]
overc_max = results_data_["overconsume"].values[np.argmax(results_data_["overconsume"].values)]

# regular grid covering the domain of the data
X,Y = np.meshgrid(np.arange(underc_min, underc_max, 100), np.arange(overc_min, overc_max, 100))
XX = X.flatten()
YY = Y.flatten()
# best-fit quadratic curve
A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2]
C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
# evaluate it on a grid
Z = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX*YY, XX**2, YY**2], C).reshape(X.shape)

# results_data_complete = results_data
# results_data = results_data.iloc[:-6]
# results_data.info()

In [18]:
fig = px.scatter_3d(results_data, x='underconsume', y='overconsume', z='av_EV_energy_left',
                    #size='scaled_cumulative_reward',
                    hover_data=['cumulative_reward'],
                    color='Name',
                    labels={
                     "av_EV_energy_left": "Av.EVs battery at departure(kWh)",
                     "Name": "number of weekly EVs arrival test",
                     "underconsume": "Unused RE-to-vehicle energy(kWh)",
                     "overconsume": "Grid energy used(kWh)"
                    })
#fig.update_layout(scene_zaxis_type="log")
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
#fig.add_trace(go.Surface(x=X, y=Y, z=Z, showscale=False, opacity=0.5))
fig.update_layout(title_text='EVs arrivals experiment results scatter plot',  width=1400,height=700,)
# fig.update_layout(
#     scene = dict(
#         xaxis = dict(nticks=4, range=[-100,100],),
#                      yaxis = dict(nticks=4, range=[-50,100],),
#                      zaxis = dict(nticks=4, range=[-100,100],),),
#     width=700,
#     margin=dict(r=20, l=10, b=10, t=10))
#fig.show()

In [19]:
min_rew = results_data["cumulative_reward"].values[np.argmin(results_data["cumulative_reward"].values)]
max_rew = results_data["cumulative_reward"].values[np.argmax(results_data["cumulative_reward"].values)]
results_data["scaled_cumulative_reward"] = np.interp(results_data["cumulative_reward"].values, [min_rew, max_rew], [1, 50] )

fig = px.scatter_3d(results_data, x='underconsume', y='overconsume', z='av_EV_energy_left',
                    size='scaled_cumulative_reward',
                    hover_data=['cumulative_reward'],
                    color='Name',
                    labels={
                     "av_EV_energy_left": "Av.EVs battery at departure(kWh)",
                     "Name": "number of weekly EVs arrival test",
                     "underconsume": "Unused RE-to-vehicle energy(kWh)",
                     "overconsume": "Grid energy used(kWh)"
                    })
#fig.update_layout(scene_zaxis_type="log")
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
#fig.add_trace(go.Surface(x=X, y=Y, z=Z, showscale=False, opacity=0.3))
fig.update_layout(title_text='EVs arrivals experiment results scatter plot (size being the scaled_cumulative_reward)',  width=1400,height=700,)

In [20]:
fig = px.scatter_3d(results_data, x='underconsume', y='overconsume', z='av_EV_energy_left',
                    #size='scaled_cumulative_reward',
                    color_continuous_scale=px.colors.sequential.Viridis,
                    hover_data=['cumulative_reward','Name'],
                    color='cumulative_reward',
                    labels={
                     "av_EV_energy_left": "Av.EVs battery at departure(kWh)",
                     "Name": "Algorithm type",
                     "underconsume": "Unused RE-to-vehicle energy(kWh)",
                     "overconsume": "Grid energy used(kWh)"
                    })
#fig.update_layout(scene_zaxis_type="log")
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
#fig.add_trace(go.Surface(x=X, y=Y, z=Z, showscale=False, opacity=0.3))
fig.update_layout(title_text='EVs arrivals experiment results scatter plot',  width=1400,height=700,)

In [21]:
from sklearn.linear_model import LinearRegression
min_overc = results_data["overconsume"].values[np.argmin(results_data["overconsume"].values)]
max_overc = results_data["overconsume"].values[np.argmax(results_data["overconsume"].values)]
# Get index for the second highest value.
min_overc2 = results_data["overconsume"].values[results_data["overconsume"].values.argsort()[1]]
#print(min_overc2)
results_data["inverted_overconsume"] = np.interp(results_data["overconsume"].values, [min_overc, min_overc2, max_overc], [1, 1, 0] )

fig = px.scatter(results_data, x='underconsume', y='av_EV_energy_left',
                    #color = 'scaled_cumulative_reward',
                    #color = 'inverted_overconsume',
                    #color_continuous_scale=px.colors.sequential.Viridis,
                    #size = 'scaled_cumulative_reward',
                    size='inverted_overconsume',
                    hover_data=['cumulative_reward','overconsume'],
                    color='Name',
                    #text="Name",
                    labels={
                     "av_EV_energy_left": "Av.EVs battery at departure(kWh)",
                     "Name": "number of weekly EVs arrival test",
                     "underconsume": "Unused RE-to-vehicle energy(kWh)",
                     "overconsume": "Grid energy used(kWh)",
                     "inverted_overconsume": "inverted Grid energy used[0-1]"
                    })

fig.update_layout(title_text=f'EVs arrivals experiment results scatter plot (size being the mapped Grid energy used [{round(min_overc/1000,1)}, {round(max_overc/1000,3)} MWh]-->[1,0])',  width=1600,height=800,)

In [22]:
fig = px.scatter(results_data, x='underconsume', y='av_EV_energy_left',
                    color = 'cumulative_reward',
                    #color = 'inverted_overconsume',
                    color_continuous_scale=px.colors.sequential.Viridis,
                    #size = 'scaled_cumulative_reward',
                    size='inverted_overconsume',
                    hover_data=['cumulative_reward','overconsume'],
                    #color='Name',
                    #text="Name",
                    labels={
                     "av_EV_energy_left": "Av.EVs battery at departure(kWh)",
                     "Name": "number of weekly EVs arrival test",
                     "underconsume": "Unused RE-to-vehicle energy(kWh)",
                     "overconsume": "Grid energy used(kWh)",
                     "inverted_overconsume": "inverted Grid energy used[0-1]"
                    })

fig.update_layout(title_text=f'EVs arrivals experiment results scatter plot (size being the mapped Grid energy used [{round(min_overc/1000,1)}, {round(max_overc/1000,3)} MWh]-->[1,0])',  width=1500,height=800,)

In [23]:
min_overc = results_data["overconsume"].values[np.argmin(results_data["overconsume"].values)]
max_overc = results_data["overconsume"].values[np.argmax(results_data["overconsume"].values)]
# Get index for the second highest value.
min_overc2 = results_data["overconsume"].values[results_data["overconsume"].values.argsort()[1]]
#print(min_overc2)
results_data["inverted_overconsume"] = np.interp(results_data["overconsume"].values, [min_overc, min_overc2, max_overc], [1, 1, 0] )

# selecting rows based on condition
filtered_data = results_data.loc[results_data['inverted_overconsume'] > 0.98]

x = filtered_data['underconsume'].values
x = x.reshape((-1,1))
y = filtered_data['av_EV_energy_left'].values
model = LinearRegression().fit(x, y)
x_plot = np.arange(25000)
x_new = x_plot.reshape((-1, 1))
y_new = model.predict(x_new)

fig = px.scatter(results_data, x='underconsume', y='av_EV_energy_left',
                    #color = 'scaled_cumulative_reward',
                    color = 'overconsume',
                    #color_continuous_scale=px.colors.sequential.Viridis,
                    size = 'scaled_cumulative_reward',
                    #size='inverted_overconsume',
                    hover_data=['cumulative_reward','overconsume'],
                    #color='Name',
                    #text="Name",
                    labels={
                     "av_EV_energy_left": "Av.EVs battery at departure(kWh)",
                     "Name": "Algorithm type",
                     "underconsume": "Unused RE-to-vehicle energy(kWh)",
                     "overconsume": "Grid energy used(kWh)",
                     "inverted_overconsume": "inverted Grid energy used(0-1)"
                    })


#fig.update_layout(scene_zaxis_type="log")
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
#fig.add_trace(go.Scatter(x=x_plot, y=y_new, line={'color':'yellow'},
#                            #name="minimum overconsume regression line"
#                            ))
fig.update_layout(title_text='EVs arrivals experiment results scatter plot (size being the scaled_cumulative_reward)',  width=1600,height=800,)